Antes de empezar: un poco de documentación para enterder ligeramente cómo funciona HTML:

https://www.w3schools.com/html/html_intro.asp

# Web Scraping: Selenium

En este notebook veremos cómo automatizar la extracción de datos de una página web simulando la navegación. Para ello utilizaremos la librería Selenium:
https://selenium-python.readthedocs.io/

El software libre Selenium es un **framework** para realizar test automatizados de software a aplicaciones web. En principio, fue desarrollado para poner a prueba páginas y apps web, pero el **WebDriver** de Selenium también puede usarse con Python para realizar scraping. Si bien Selenium en sí no está escrito en Python, con este lenguaje de programación es posible acceder a las funciones del software.

A diferencia de Scrapy y de BeautifulSoup, Selenium no trabaja con el texto fuente en HTML de la web en cuestión, sino que carga la página en un navegador sin interfaz de usuario. El navegador interpreta entonces el código fuente de la página y crea, a partir de él, un **Document Object Model** (modelo de objetos de documento o DOM). Esta interfaz estandarizada permite poner a prueba las interacciones de los usuarios. De esta forma se consigue, por ejemplo, simular clics y rellenar formularios automáticamente. Los cambios en la web que resultan de dichas acciones se reflejan en el DOM. La estructura del proceso de web scraping con Selenium es la siguiente:

URL → Solicitud HTTP → HTML → Selenium → DOM

In [1]:
# Imports
import pandas as pd
from selenium import webdriver

/home/jose/.local/lib/python3.7/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


Deberemos descargar un archivo Chrome driver para selenium en https://chromedriver.chromium.org/downloads, es importante que compruebes la versión de Chrome que utilizas y descargaues el driver correspondiente.
Una vez descargado crearemos una variable con la dirección completa del path al chromedriver 

El objeto driver es con el que trabajaremos a partir de ahora

In [79]:
# especificamos el path hasta nuestro driver recién descargado:
chrome_driver_path = '/home/jose/Descargas/chromedriver_linux64/chromedriver'
options =webdriver.ChromeOptions()
# headless nos permite no tener que ver la navegación en la ventana de Chrome
#option.add_argument('headless')
options.add_argument('--disable-extensions')

In [83]:
# Creamos el driver con el que nos vamos a manejar en la sesión de scrapeo:
driver = webdriver.Chrome(executable_path=chrome_driver_path, options=options)

In [84]:
# indicamos la URL de la página web a la que queremos acceder
url = 'https://www.filmaffinity.com/es/main.html'
# el objeto driver nos va a permitir alterar el estado del la página
driver.get(url)

¡Vaya! Nos hemos encontrado con un pop-up que nos pide aceptar cookies o algo po el estilo, si nos lo queremos saltar, para poder automatizar el proceso deberemos:

1. Buscamos el botón
2. Hacemos click en el botón

Selenium nos permite buscar elementos por etiqueteas de html: https://selenium-python.readthedocs.io/locating-elements.html

CUIDADO! Los metodos de busqueda de elementos estan "duplicados". Tenemos *find_element_by_tag_name* y *find_elements_by_tag_name* el primero nos devolverá el primer elemento que encuentre (aunque haya más), el segundo nos devolverá una lista con todos los elementos que encuentre (aunque solo sea uno)

In [85]:
# Navegando en filmaffinity, en el pop-up:
element_by_tag = driver.find_elements_by_tag_name('button')
element_by_xpath = driver.find_element_by_xpath('/html/body/div[1]/div/div/div/div[2]/div/button[2]')
print(type(element_by_tag))
print(type(element_by_xpath))
print(element_by_tag)
print(element_by_xpath)

<class 'list'>
<class 'selenium.webdriver.remote.webelement.WebElement'>
[<selenium.webdriver.remote.webelement.WebElement (session="e03aa374b7ab06d033347c3499f63b55", element="61833fdf-f030-40eb-af54-293416f439f9")>, <selenium.webdriver.remote.webelement.WebElement (session="e03aa374b7ab06d033347c3499f63b55", element="de28df42-64ba-4e34-8a7e-559d0c147cc5")>, <selenium.webdriver.remote.webelement.WebElement (session="e03aa374b7ab06d033347c3499f63b55", element="5c7e313e-4ca3-4bf8-9cbd-f71e5a0783a7")>]
<selenium.webdriver.remote.webelement.WebElement (session="e03aa374b7ab06d033347c3499f63b55", element="5c7e313e-4ca3-4bf8-9cbd-f71e5a0783a7")>


Una vez tenemos los elementos podemos hacer varias cosas con ellos

Podemos extraer todos los atributos que tenga

In [6]:
print(element_by_xpath.size)
print(element_by_tag[0].tag_name)
dir(element_by_xpath)

{'height': 40, 'width': 340}
button


['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_execute',
 '_id',
 '_parent',
 '_upload',
 '_w3c',
 'clear',
 'click',
 'find_element',
 'find_element_by_class_name',
 'find_element_by_css_selector',
 'find_element_by_id',
 'find_element_by_link_text',
 'find_element_by_name',
 'find_element_by_partial_link_text',
 'find_element_by_tag_name',
 'find_element_by_xpath',
 'find_elements',
 'find_elements_by_class_name',
 'find_elements_by_css_selector',
 'find_elements_by_id',
 'find_elements_by_link_text',
 'find_elements_by_name',
 'find_elements_by_partial_link_text',
 'find_elements_by_tag_name',
 'find_elements_by_xpath',
 'get_attribute',
 'get_property',
 'id',
 

In [7]:
for pos, elemen in enumerate(element_by_tag):
    elemen.screenshot(elemen.text + str(pos) + '.png')

Podemos evaluar que tipo de elemento es (tag)

In [86]:
boton_aceptar = element_by_tag[2]
boton_aceptar.click()

Podemos sacar el valor que tiene (el texto)

Incluso podemos guardar una imagen del elemento

In [73]:
# Vemos que es tipo 'WebElement' y en la documentación podremos encontrar sus métodos

In [74]:
# guardamos como 'mi_imagen.png' la imagen asociada al xpath


Evaluamos que elementos hemos encontrado por el tag:

Nos quedamos con el botón que nos interesa

Si el elemento es interactivo podremos hacer más cosas además de las anteriores. Por ejemplo: hacer click

Buscamos una película por título

In [75]:
from selenium.webdriver.common.keys import Keys

In [78]:
buscador = driver.find_element_by_xpath('/html/body/div[2]/div[1]/div/div[2]/form/input[1]')

In [79]:
buscador.send_keys('Interstellar')
buscador.send_keys(Keys.ENTER)

In [12]:
# una vez escrita la búsqueda deberíamos poder activarla:
menu_lateral = driver.find_element_by_id('lsmenu')
secciones_menu = menu_lateral.find_elements_by_tag_name('a')

In [14]:
# volvemos a la página anterior
for elemen in secciones_menu:
    if elemen.text == 'Próximos estrenos':
        elemen.click()     

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=90.0.4430.212)


### Vamos a buscar todas las películas que se estrenan el próximo viernes

1. Cogemos los containers que hay en la zona central

2. Vemos con cuál nos tenemos que quedar

Accedemos al container central, en el que aparecen los estrenos por semana que queremos ver, exactamente igual que hemos hecho antes

In [23]:
cajon_central = driver.find_elements_by_id("main-wrapper-rdcat")
for semana in cajon_central:
    print(semana.find_element_by_tag_name('div').text)
    print(semana.find_element_by_tag_name('div').get_attribute('id'))

21 de mayo de 2021
2021-05-21
28 de mayo de 2021
2021-05-28
3 de junio de 2021
2021-06-03
4 de junio de 2021
2021-06-04
11 de junio de 2021
2021-06-11
16 de junio de 2021
2021-06-16
18 de junio de 2021
2021-06-18
25 de junio de 2021
2021-06-25


In [25]:
for semana in cajon_central:
    fecha = semana.find_element_by_tag_name('div').get_attribute('id')
    if fecha == '2021-05-21':
        break

In [29]:
caratula = semana.find_elements_by_class_name('mc-poster')
lista_pelis = []
for pelis in caratula:
    lista_pelis.append(pelis.find_element_by_tag_name('a').get_attribute('href'))
lista_pelis

['https://www.filmaffinity.com/es/film637149.html',
 'https://www.filmaffinity.com/es/film478891.html',
 'https://www.filmaffinity.com/es/film713795.html',
 'https://www.filmaffinity.com/es/film112106.html',
 'https://www.filmaffinity.com/es/film329087.html',
 'https://www.filmaffinity.com/es/film950177.html',
 'https://www.filmaffinity.com/es/film388427.html',
 'https://www.filmaffinity.com/es/film132801.html',
 'https://www.filmaffinity.com/es/film759533.html',
 'https://www.filmaffinity.com/es/film407520.html',
 'https://www.filmaffinity.com/es/film452148.html',
 'https://www.filmaffinity.com/es/film158082.html',
 'https://www.filmaffinity.com/es/film108823.html']

In [37]:
driver.get(lista_pelis[0])
titulo = driver.find_element_by_xpath('/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[3]/dl[1]/dd[1]').text
nota = driver.find_element_by_xpath('/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[2]/div[2]/div[1]/div[2]/div[1]').get_attribute('content')
votos = driver.find_element_by_xpath('/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[2]/div[2]/div[1]/div[2]/div[2]/span').get_attribute('content')
print(titulo, nota, votos)

Spiral: From the Book of Saw 4.9 39


In [40]:
fichas  = driver.find_element_by_xpath('/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[3]/dl[1]')

In [41]:
ficha_clave = []
ficha_valores = []

for name in fichas.find_elements_by_tag_name('dt'):
    ficha_clave.append(name.text)
for name in fichas.find_elements_by_tag_name('dd'):
    ficha_valores.append(name.text)

In [46]:
columns = ['Titulo', 'Nota', 'Votos']
columns.extend(ficha_clave)
columns
values = [titulo, nota, votos]
values.extend(ficha_valores)

In [47]:
ficha_valores

['Spiral: From the Book of Saw',
 '2021',
 '93 min.',
 ' Estados Unidos',
 'Darren Lynn Bousman',
 'Josh Stolberg, Pete Goldfinger. Historia: Pete Goldfinger, Chris Rock, Josh Stolberg',
 'Charlie Clouser',
 'Jordan Oram',
 'Chris Rock, Samuel L. Jackson, Max Minghella, Marisol Nichols, Zoie Palmer, Nazneen Contractor, Dan Petronijevic, Morgan David Jones, Edie Inksetter, K.C. Collins, Josh Stolberg, Ali Johnson, Brian Cook, ver 6 más',
 'Serendipity Productions, Twisted Pictures, Burg/Koules Productions, Dahlstar. Distribuidora: Lionsgate',
 'Terror. Thriller | Policíaco. Crimen. Asesinos en serie',
 'Saw',
 'Trabajando a la sombra de un policía veterano (Samuel L. Jackson), el intrépido detective Ezekiel “Zeke” Banks (Chris Rock) y su compañero novato (Max Minghella) toman las riendas de una investigación relacionada con una serie de asesinatos que albergan ciertos paralelismos con el sangriento pasado de la ciudad. A medida que profundiza en el misterio, Zeke descubre que él mismo s

In [49]:
pd.DataFrame([values], columns=columns)

,Titulo,Nota,Votos,Título original,Año,Duración,País,Dirección,Guion,Música,Fotografía,Reparto,Productora,Género,Grupos,Sinopsis
0,Spiral: From the Book of Saw,4.9,39,Spiral: From the Book of Saw,2021,93 min.,Estados Unidos,Darren Lynn Bousman,"Josh Stolberg, Pete Goldfinger. Historia: Pete...",Charlie Clouser,Jordan Oram,"Chris Rock, Samuel L. Jackson, Max Minghella, ...","Serendipity Productions, Twisted Pictures, Bur...",Terror. Thriller | Policíaco. Crimen. Asesinos...,Saw,Trabajando a la sombra de un policía veterano ...


In [31]:
driver.back()

Buscamos cómo acceder a las películas

In [74]:
def sacar_info(driver):
    driver.get(lista_pelis[0])
    titulo = driver.find_element_by_xpath('/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[3]/dl[1]/dd[1]').text
    nota = driver.find_element_by_xpath('/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[2]/div[2]/div[1]/div[2]/div[1]').get_attribute('content')
    votos = driver.find_element_by_xpath('/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[2]/div[2]/div[1]/div[2]/div[2]/span').get_attribute('content')
    fichas  = driver.find_element_by_xpath('/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[3]/dl[1]')

    return titulo, nota, votos, fichas

def sacar_ficha(fichas):
    ficha_clave = []
    ficha_valores = []

    for name in fichas.find_elements_by_tag_name('dt'):
        ficha_clave.append(name.text)
    for name in fichas.find_elements_by_tag_name('dd'):
        ficha_valores.append(name.text)
    return ficha_clave, ficha_valores

def montar_serie(ficha_clave, ficha_valores, titulo, nota, votos):
    columns = ['Titulo', 'Nota', 'Votos']
    columns.extend(ficha_clave)
    values = [titulo, nota, votos]
    values.extend(ficha_valores)

    return pd.DataFrame([values], columns=columns)

def montar_df(driver):
    titulo, nota, votos, ficha = sacar_info(driver)
    ficha_claves, ficha_valores = sacar_ficha(ficha)
    montar_serie = montar_serie(ficha_clave, ficha_valores, titulo, nota, votos)

    return

In [52]:
montar_df(driver)

NameError: name 'ficha' is not defined

Una vez tenemos todas las urls vamos a ver qué hacemos con cada una de ellas

In [ ]:
# Accedemos a la página de la primera pelicula


Vamos a ver el proceso que deberíamos hacer con cada una de las películas:

1. Sacamos toda la información que nos interesa

In [54]:
# titulo, nota, numero de votos y ficha técnica
driver.execute_script('window.open("")')

In [58]:
driver.switch_to.window(driver.window_handles[-1])

In [57]:
driver.close()

2. Creamos una lista a partir de la ficha técnica

In [ ]:
# Los nombres estan con tag  = 'dt' y los valores con 'dd'


3. Creamos un dataframe con la info

Ahora vamos a crear una función que nos haga todo esto para cada una de las películas:

## Modo Dios: moviendonos entre ventanas

Vamos a ver cómo nos podemos mover entre ventanas del navegador

Abrir nueva ventana:

Movernos a otra ventana

Cerrar ventana

Una vez cerramos la ventana tenemos que indicarle a qué ventana tiene que ir

Sabiendo cómo podemos movernos por entre las ventanas y sabiendo cómo extraer de cada página toda la información que necesitamos vamos a crear nuestro dataframe:

In [60]:
# para abrir todos los links en lista_pelis
for pelis in lista_pelis:
    driver.execute_script(f"window.open('{pelis}')")
    driver.switch_to.window(driver.window_handles[-1])
    driver.get(pelis)
    pass

In [78]:
# Creamos un dataframe con todas las pelis que se estrenan la próxima semana:



UnboundLocalError: local variable 'montar_serie' referenced before assignment

In [87]:
def sacar_info(driver):
    titulo = driver.find_element_by_xpath('/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/h1/span').text
    try:
        nota = driver.find_element_by_xpath('/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[2]/div[2]/div[1]/div[2]/div[1]').text
        votos = driver.find_element_by_xpath('/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[2]/div[2]/div[1]/div[2]/div[2]/span').text
    except:
        nota = None
        votos = None
    ficha = driver.find_element_by_xpath('/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div')
    return titulo, nota, votos, ficha

def sacar_ficha(ficha):
    ficha_claves = []
    ficha_valores = []

    for i, name in enumerate(ficha.find_elements_by_tag_name('dt')):
        ficha_claves.append(name.text)
        ficha_valores.append(ficha.find_elements_by_tag_name('dd')[i].text)
    
    return ficha_claves, ficha_valores


def montar_df(ficha_claves, ficha_valores, titulo, nota, votos):
    columns = ['Titulo', 'Nota', 'Votos']
    values = [titulo, nota, votos]
    
    columns.extend(ficha_claves)
    values.extend(ficha_valores)
    
    return pd.DataFrame([values], columns = columns)

def nueva_peli(driver):
    titulo, nota, votos, ficha = sacar_info(driver)
    ficha_claves, ficha_valores = sacar_ficha(ficha)
    df_peli = montar_df(ficha_claves, ficha_valores, titulo, nota, votos) 
    
    return df_peli

df_peliculas = pd.DataFrame() 
for pelis in lista_pelis:
    driver.execute_script(f"window.open('{pelis}')")
    driver.switch_to.window(driver.window_handles[-1])
    driver.get(pelis)
    nuevas_peli = nueva_peli(driver)
    df_peliculas = df_peliculas.append(nuevas_peli)
    pass

In [88]:
df_peliculas

,Titulo,Nota,Votos,Título original,Año,Duración,País,Dirección,Guion,Música,...,Género,Grupos,Sinopsis,Críticas,Tu crítica,Votaciones de almas gemelas,Votaciones de tus amigos,Posición rankings listas,Premios,
0,Spiral: Saw,"4,9",40,Spiral: From the Book of Saw,2021,93 min.,Estados Unidos,Darren Lynn Bousman,"Josh Stolberg, Pete Goldfinger. Historia: Pete...",Charlie Clouser,...,Terror. Thriller | Policíaco. Crimen. Asesinos...,Saw,Trabajando a la sombra de un policía veterano ...,"""La saga de 'Saw' no ha cambiado nada. Así que...",Escribe tu opinión para que el resto de los us...,Regístrate y podrás acceder a recomendaciones ...,Regístrate y podrás acceder a todas las votaci...,99 Próximos estrenos: éstas no me las pierdo (...,NaN,NaN
0,Poliamor para principiantes,"4,4",26,Poliamor para principiantes,2021,97 min.,España,Fernando Colomo,"Fernando Colomo, Casandra Macías Gago, Marina ...",Fernando Furones,...,Comedia. Romance | Comedia romántica,NaN,Satur (Karra Elejalde) y Tina (Toni Acosta) so...,NaN,Escribe tu opinión para que el resto de los us...,Regístrate y podrás acceder a recomendaciones ...,Regístrate y podrás acceder a todas las votaci...,NaN,NaN,NaN
0,Érase una vez en Queens,None,None,Boogie,2021,89 min.,Estados Unidos,Eddie Huang,Eddie Huang,"Ali Shaheed Muhammad, Adrian Younge",...,Drama | Baloncesto. Inmigración,NaN,"En la Nueva York contemporánea, un joven fenóm...","""Mientras que su acercamiento a los personajes...",Escribe tu opinión para que el resto de los us...,Regístrate y podrás acceder a recomendaciones ...,Regístrate y podrás acceder a todas las votaci...,NaN,NaN,NaN
0,First Cow,"6,9",1.396,First Cow,2019,121 min.,Estados Unidos,Kelly Reichardt,"Jonathan Raymond, Kelly Reichardt. Novela: Jon...",William Tyler,...,Drama. Western | Siglo XIX. Amistad,NaN,Narra la historia de un cocinero (John Magaro)...,"""Una película tan plena de humor como de trist...",Escribe tu opinión para que el resto de los us...,Regístrate y podrás acceder a recomendaciones ...,Regístrate y podrás acceder a todas las votaci...,53 Mis películas favoritas del 2020 (24),2020: National Board of Review (NBR): Top 10 p...,NaN
0,Mía y Moi,"6,0",67,Mía y Moi,2021,NaN,España,Borja de la Vega,Borja de la Vega,NaN,...,Drama,NaN,"Tras perder a su madre, Mía y Moi se refugian ...","""Hay películas ultrasensoriales, que se huelen...",Escribe tu opinión para que el resto de los us...,Regístrate y podrás acceder a recomendaciones ...,Regístrate y podrás acceder a todas las votaci...,NaN,NaN,NaN
0,Pequeño país,"6,4",30,Petit pays aka,2020,111 min.,Francia,Eric Barbier,Eric Barbier. Novela: Gaël Faye,Renaud Barbier,...,Drama,NaN,Gabriel tiene 10 años y vive en un cómodo veci...,"""Un vehículo entrañable y pedagógico, un buen ...",Escribe tu opinión para que el resto de los us...,Regístrate y podrás acceder a recomendaciones ...,Regístrate y podrás acceder a todas las votaci...,NaN,2020: Premios César: Nominada a mejor guion ad...,
0,Sweat,"6,5",44,Sweat,2020,100 min.,Polonia,Magnus von Horn,Magnus von Horn,Piotr Kurek,...,Drama | Internet/Informática,NaN,Describe tres días en la vida de la motivadora...,"""Pocas dudas hay del poderoso influjo del maes...",Escribe tu opinión para que el resto de los us...,Regístrate y podrás acceder a recomendaciones ...,Regístrate y podrás acceder a todas las votaci...,NaN,2020: Festival internacional de Chicago: Mejor...,NaN
0,Félix y el tesoro de Morgäa,None,None,Félix et le trésor de Morgäa aka,2021,90 min.,Canadá,Nicola Lemay,Marc Robitaille,NaN,...,Animación. Aventuras | Adolescencia,NaN,Félix es un niño de 12 años que está convencid...,NaN,Escribe tu opinión para que el resto de los us...,Regístrate y podrás acceder a recomendaciones ...,Regístrate y podrás acceder a todas las votaci...,NaN,NaN,
0,El viaje de Chihiro,"8,1",109.289,Sen to Chihiro no kamikakushi aka,2001,124 min.,Japón,Hayao Miyazaki,Hayao Miyazaki,Joe Hisaishi,...,Animación. Fantástico. Aventuras | Dragones. P...,Studio Ghibli (y obras rela

¡Tachán! Ya tenemos un dataframe con todas las películas que se van a estrenar el próximo viernes